<a href="https://colab.research.google.com/github/johanjun/Google_Machine_Learning_Bootcamp_KR_2021/blob/main/kaggle/RSNA_MICCAI_BRAIN_TUMOR/rsna_miccai_brain_tumor_johanjun_1006_T1wCE_T2w_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pydicom

     |████████████████████████████████| 2.0 MB 7.8 MB/s 


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm
import random
import pydicom
import cv2
import pickle

import matplotlib.pyplot as plt
import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
data_dir = '/content/drive/MyDrive/Google_Machine_Learning_Bootcamp_KR_2021/RSNA_MICCAI/data/rsna-miccai/'

In [5]:
train_df = pd.read_csv(data_dir + "train_labels.csv")
test_df = pd.read_csv(data_dir + "sample_submission.csv")

In this discussion a competition host has notified that there are some issues with these 3 cases
Patient IDs -

1. 00109 (FLAIR images are blank)
2. 00123 (T1w images are blank)
3. 00709 (FLAIR images are blank)
<br><br>Hence these can be excluded

In [6]:
#refer: https://www.kaggle.com/arnabs007/part-1-rsna-miccai-btrc-understanding-the-data
EXCLUDE = [109, 123, 709]
train_df = train_df[~train_df.BraTS21ID.isin(EXCLUDE)]

In [7]:
TYPES = ["FLAIR", "T1w", "T1wCE", "T2w"] #mpMRI scans

In [8]:
def load_dicom(path, size = 128): #load DICOM files
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array #returns a numpy.ndarray containing the pixel data
    if np.max(data) != 0:
        data = data / np.max(data) #standardizes so that the pixel values are between 0 and 1
    data = (data * 255).astype(np.uint8) #rescales to 0 and 255
    return cv2.resize(data, (size, size))

In [9]:
def get_all_image_paths(BraTS21ID, image_type, folder="train"): #get an array of all the images of a particular type or a particular patient id
    assert(image_type in TYPES) #only in types
    patient_path = os.path.join(data_dir + "%s/" % folder, str(BraTS21ID).zfill(5)) #다른 폴더일 수도 있음
    #print(lambda x: int(x[:-4].split("-")[-1]))
    
    paths = sorted(glob.glob(os.path.join(patient_path, image_type, "*")), key=lambda x: int(x[:-4].split("-")[-1])) #sort
    #print(paths)
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)
    if num_images < 10:
        jump = 1
    else:
        jump = 3
        
    return np.array(paths[start:end:jump])

In [10]:
def get_all_images(BraTS21ID, image_type, folder="train", size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(BraTS21ID, image_type, folder)]

In [11]:
def get_all_data_train(image_type):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        tmp_x = train_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "train", IMAGE_SIZE)
        label = tmp_x["MGMT_value"]

        X += images
        y += [label] * len(images)
        train_ids += [int(tmp_x["BraTS21ID"])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

def get_all_data_test(image_type):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        tmp_x = test_df.loc[i]
        images = get_all_images(int(tmp_x["BraTS21ID"]), image_type, "test", IMAGE_SIZE)
        X += images
        test_ids += [int(tmp_x["BraTS21ID"])] * len(images)

    return np.array(X), np.array(test_ids)

In [12]:
#pickling
with open(data_dir+'X_T1wCE.pkl', 'rb') as f:
  X_T1wCE = pickle.load(f)

with open(data_dir+'y_T1wCE.pkl', 'rb') as f:
  y_T1wCE = pickle.load(f)

with open(data_dir+'train_idt_T1wCE.pkl', 'rb') as f:
  train_idt_T1wCE = pickle.load(f)

with open(data_dir+'X_test_T1wCE.pkl', 'rb') as f:
  X_test_T1wCE = pickle.load(f)

with open(data_dir+'test_idt_T1wCE.pkl', 'rb') as f:
  test_idt_T1wCE = pickle.load(f)
#pickling
with open(data_dir+'X_FLAIR.pkl', 'rb') as f:
  X_FLAIR = pickle.load(f)

with open(data_dir+'y_FLAIR.pkl', 'rb') as f:
  y_FLAIR = pickle.load(f)

with open(data_dir+'train_idt_FLAIR.pkl', 'rb') as f:
  train_idt_FLAIR = pickle.load(f)

with open(data_dir+'X_test_FLAIR.pkl', 'rb') as f:
  X_test_FLAIR = pickle.load(f)

with open(data_dir+'test_idt_FLAIR.pkl', 'rb') as f:
  test_idt_FLAIR = pickle.load(f)
#pickling
with open(data_dir+'X_T2w.pkl', 'rb') as f:
  X_T2w = pickle.load(f)

with open(data_dir+'y_T2w.pkl', 'rb') as f:
  y_T2w = pickle.load(f)

with open(data_dir+'train_idt_T2w.pkl', 'rb') as f:
  train_idt_T2w = pickle.load(f)

with open(data_dir+'X_test_T2w.pkl', 'rb') as f:
  X_test_T2w = pickle.load(f)

with open(data_dir+'test_idt_T2w.pkl', 'rb') as f:
  test_idt_T2w = pickle.load(f)
#pickling
with open(data_dir+'X_T1w.pkl', 'rb') as f:
  X_T1w = pickle.load(f)

with open(data_dir+'y_T1w.pkl', 'rb') as f:
  y_T1w = pickle.load(f)

with open(data_dir+'train_idt_T1w.pkl', 'rb') as f:
  train_idt_T1w = pickle.load(f)

with open(data_dir+'X_test_T1w.pkl', 'rb') as f:
  X_test_T1w = pickle.load(f)

with open(data_dir+'test_idt_T1w.pkl', 'rb') as f:
  test_idt_T1w = pickle.load(f)

In [13]:
X = np.concatenate([X_T1wCE, X_T2w], axis=0)
y = np.concatenate([y_T1wCE, y_T2w], axis=0)
train_idt = np.concatenate([train_idt_T1wCE, train_idt_T2w], axis=0)

X_test = np.concatenate([X_test_FLAIR, X_test_T1w, X_test_T1wCE, X_test_T2w], axis=0)
test_idt = np.concatenate([test_idt_FLAIR, test_idt_T1w, test_idt_T1wCE, test_idt_T2w], axis=0)

In [14]:
X_train, X_valid, y_train, y_valid, train_idt_train, train_idt_valid = train_test_split(X, y, train_idt, test_size=0.075)

In [15]:
X_train = tf.expand_dims(X_train, axis=-1) #expand the dimension at the end of the array
X_valid = tf.expand_dims(X_valid, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

y_train = to_categorical(y_train) #one-hot incoding
y_valid = to_categorical(y_valid)

In [16]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, train_idt_train.shape, train_idt_valid.shape

(TensorShape([30533, 64, 64, 1]),
 (30533, 2),
 TensorShape([2476, 64, 64, 1]),
 (2476, 2),
 (30533,),
 (2476,))

In [17]:
X2 = np.concatenate([X_FLAIR, X_T2w, X_T1w, X_T1wCE], axis=0)
y2 = np.concatenate([y_FLAIR, y_T2w, y_T1w, y_T1wCE], axis=0)
train_idt2 = np.concatenate([train_idt_FLAIR, train_idt_T2w, train_idt_T1w, train_idt_T1wCE], axis=0)

In [18]:
X_train2, X_valid2, y_train2, y_valid2, train_idt_train2, train_idt_valid2 = train_test_split(X2, y2, train_idt2, test_size=0.05)

In [19]:
X_train2 = np.expand_dims(X_train2, axis=-1) 
X_valid2 = np.expand_dims(X_valid2, axis=-1)

y_train2 = to_categorical(y_train2) 
y_valid2 = to_categorical(y_valid2)

In [30]:
# %load train_lib.py

def train():
    import wandb
    config_defaults = {
        'N_EPOCHS' : 15,
        'N_BATCH' : 32,
        'DENSE' : 256,
        'DROUPOUT' : 0.1
    }
    wandb.init(project='RSNA_1006_T1wCET2w',
               config=config_defaults,
               magic=True)

    config = wandb.config
    

    IMAGE_SIZE = 64
    N_EPOCHS = config.N_EPOCHS
    N_BATCH = config.N_BATCH
    N_TRAIN = X_train.shape[0]
    N_VAL = X_valid.shape[0]
    steps_per_epoch = N_TRAIN / N_BATCH
    validation_steps = int(np.ceil(N_VAL / N_BATCH))

    def create_model():
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', padding='VALID', 
                                    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      
      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='VALID'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      

      model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='SAME'))
      model.add(tf.keras.layers.BatchNormalization())
      model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
      model.add(tf.keras.layers.Dropout(config.DROPOUT))

      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(config.DENSE, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      model.add(tf.keras.layers.Dense(2, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
      return model
    
    model = create_model()
    

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.AUC()])
    
    model.fit(
    x=X_train,
    y=y_train,
    epochs=N_EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=(X_valid2, y_valid2))

In [31]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'N_EPOCHS': {
            'values': [15, 40, 80]
        },
        'N_BATCH': {
            'values': [32, 64, 128]
        },
        'DENSE': {
            'values': [256, 1024]
        },
        'DROPOUT': {
            'values': [0.1, 0.4]
        },
    }
}

In [21]:
%pip install -q wandb

     |████████████████████████████████| 1.7 MB 7.7 MB/s 
     |████████████████████████████████| 139 kB 49.7 MB/s 
     |████████████████████████████████| 180 kB 53.5 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [24]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [32]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='RSNA_1006_T1wCET2w')

Create sweep with ID: 1860ugiz
Sweep URL: https://wandb.ai/johanjunme/RSNA_1006_T1wCET2w/sweeps/1860ugiz


In [33]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: jfrbonmr with config:
wandb: 	DENSE: 256
wandb: 	DROPOUT: 0.1
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 15
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/15
954/954 [==============================] - 26s 9ms/step - loss: 1.1492 - auc: 0.6250 - val_loss: 0.6784 - val_auc: 0.6520
Epoch 2/15
954/954 [==============================] - 8s 8ms/step - loss: 0.6454 - auc: 0.6857 - val_loss: 0.6632 - val_auc: 0.6467
Epoch 3/15
954/954 [==============================] - 8s 8ms/step - loss: 0.5919 - auc: 0.7448 - val_loss: 0.6830 - val_auc: 0.6467
Epoch 4/15
954/954 [==============================] - 8s 9ms/step - loss: 0.5283 - auc: 0.8045 - val_loss: 0.6056 - val_auc: 0.7425
Epoch 5/15
954/954 [==============================] - 8s 8ms/step - loss: 0.4703 - auc: 0.8488 - val_loss: 0.6524 - val_auc: 0.7347
Epoch 6/15
954/954 [==============================] - 8s 8ms/step - loss: 0.4260 - auc: 0.8799 - val_loss: 0.5850 - val_auc: 0.8027
Epoch 7/15
954/954 [==============================] - 8s 8ms/step - loss: 0.3735 - auc: 0.9122 - val_loss: 0.5824 - val_auc: 0.8042
Epoch 8/15
954/954 [==============================] - 8s 8ms/step - loss: 0

auc,▁▂▃▄▅▆▆▇▇▇█████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▄▃▃▃▂▂▂▂▁▁▁▁
val_auc,▁▁▁▄▄▆▆▆▇▇▇████
val_loss,█▇█▃▆▂▂▃▃▃▅▁▄▅▇
auc,0.99338
best_epoch,11
best_val_loss,0.56895
epoch,14
loss,0.1223
val_auc,0.87925


wandb: Agent Starting Run: 4qsk04c0 with config:
wandb: 	DENSE: 256
wandb: 	DROPOUT: 0.1
wandb: 	N_BATCH: 32
wandb: 	N_EPOCHS: 40
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/40
954/954 [==============================] - 10s 9ms/step - loss: 1.1420 - auc: 0.6239 - val_loss: 0.6789 - val_auc: 0.6527
Epoch 2/40
954/954 [==============================] - 8s 9ms/step - loss: 0.6436 - auc: 0.6892 - val_loss: 0.6713 - val_auc: 0.6435
Epoch 3/40
954/954 [==============================] - 8s 8ms/step - loss: 0.5907 - auc: 0.7467 - val_loss: 0.6271 - val_auc: 0.7199
Epoch 4/40
954/954 [==============================] - 8s 8ms/step - loss: 0.5267 - auc: 0.8051 - val_loss: 0.6092 - val_auc: 0.7275
Epoch 5/40
954/954 [==============================] - 8s 8ms/step - loss: 0.4707 - auc: 0.8493 - val_loss: 0.5830 - val_auc: 0.7865
Epoch 6/40
954/954 [==============================] - 8s 8ms/step - loss: 0.4138 - auc: 0.8886 - val_loss: 0.5806 - val_auc: 0.8087
Epoch 7/40
954/954 [==============================] - 8s 8ms/step - loss: 0.3684 - auc: 0.9162 - val_loss: 0.6127 - val_auc: 0.8044
Epoch 8/40
954/954 [==============================] - 8s 9ms/step - loss: 0

...

auc,▁▂▃▄▅▆▆▇▇▇██████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▃▄▅▄▆▇▇▇▇▇█▇█████████▇███▇█████████████
val_loss,█▃▂▂▃▂▁▁▁▁▃▂▂▄▃▄▄▅▄▃▅▄▅▅▅▇▇▅▆▅▆▇▅▇▇▆▅▆▅▇
auc,0.99931
best_epoch,8
best_val_loss,0.56286
epoch,39
loss,0.04389
val_auc,0.86974


wandb: Agent Starting Run: 5w4pultz with config:
wandb: 	DENSE: 1024
wandb: 	DROPOUT: 0.4
wandb: 	N_BATCH: 128
wandb: 	N_EPOCHS: 80
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/80
238/238 [==============================] - 6s 20ms/step - loss: 2.8079 - auc: 0.5893 - val_loss: 1.0771 - val_auc: 0.6228
Epoch 2/80
238/238 [==============================] - 4s 16ms/step - loss: 0.8407 - auc: 0.6574 - val_loss: 0.7406 - val_auc: 0.6535
Epoch 3/80
238/238 [==============================] - 4s 16ms/step - loss: 0.6677 - auc: 0.7079 - val_loss: 0.6838 - val_auc: 0.6532
Epoch 4/80
238/238 [==============================] - 4s 16ms/step - loss: 0.5959 - auc: 0.7603 - val_loss: 0.6491 - val_auc: 0.7180
Epoch 5/80
238/238 [==============================] - 4s 16ms/step - loss: 0.5387 - auc: 0.8072 - val_loss: 0.6245 - val_auc: 0.7403
Epoch 6/80
238/238 [==============================] - 4s 16ms/step - loss: 0.4855 - auc: 0.8448 - val_loss: 0.5832 - val_auc: 0.7624
Epoch 7/80
238/238 [==============================] - 4s 16ms/step - loss: 0.4417 - auc: 0.8744 - val_loss: 0.5946 - val_auc: 0.7637
Epoch 8/80
238/238 [==============================] - 4s 16ms/step - 

auc,▁▃▅▆▇▇██████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_auc,▁▂▄▅▇▇▇▇█████▇██▇▇██▇▇███▇██▇██▇█████▇██
val_loss,▇▃▂▁▁▁▂▃▃▄▄▅▅▆▅▆▆▇▆▅▆█▆▇▇▆█▇█▆▇█▇█▆▆▇█▇▇
auc,0.99999
best_epoch,10
best_val_loss,0.55755
epoch,79
loss,0.01186
val_auc,0.86862


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
